In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()#get(name="udacity-project")#, resource_group='aml-vm-174174')
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-175042
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-175042


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster4"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', #Standard_DS3_v2
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
       "--C": choice (0.01,0.1,1),
       "--max_iter": choice(16,32,64,128) 
   })
# ({"learning_rate":uniform(0.05,0.1),"batch_size":uniform(16,32,64,128)})### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2,slack_factor=0.1)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./training', entry_script='train.py', compute_target=cpu_cluster_name,vm_size='STANDARD_D2_V2')### YOUR CODE HERE ###
#train.py should be inside ./training folder!
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=10,policy=policy, estimator=est,)

### YOUR CODE HERE ### #primary_metric_name="AUC_weighted" results in error!! Accuracy
##accuracy can be used as well

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
pipeline_run=exp.submit(config=hyperdrive_config)
RunDetails(pipeline_run).show()
pipeline_run.get_status()
pipeline_run.wait_for_completion(show_output=True) #show_output=True don't include otherwise error

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_d17495e5-00bd-43bd-8bf9-353eaef7ec74
Web View: https://ml.azure.com/runs/HD_d17495e5-00bd-43bd-8bf9-353eaef7ec74?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-175042/workspaces/quick-starts-ws-175042&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-01-13T12:41:54.852788][API][INFO]Experiment created<END>\n""<START>[2022-01-13T12:41:55.721327][GENERATOR][INFO]Trying to sample '10' jobs from the hyperparameter space<END>\n""<START>[2022-01-13T12:41:56.374256][GENERATOR][INFO]Successfully sampled '10' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_d17495e5-00bd-43bd-8bf9-353eaef7ec74
Web View: https://ml.azure.com/runs/HD_d17495e5-00bd-43bd-8bf9-353eaef7ec74?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-175042/workspaces/quick-starts-ws-175042&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Warnings:
{
  "err

{'runId': 'HD_d17495e5-00bd-43bd-8bf9-353eaef7ec74',
 'target': 'cpucluster4',
 'status': 'Canceled',
 'startTimeUtc': '2022-01-13T12:41:54.585815Z',
 'endTimeUtc': '2022-01-13T12:47:28.093656Z',
 'services': {},
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null,\n    "additionalInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'proper

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run=pipeline_run.get_best_run_by_primary_metric()
# gives error-suggests that the best_run has the value equivalence of null in other languages, which means it des not contain 
#anything. This means that the there was no result from pipeline_run.get_best_run_by_primary_metric() as used in the notebook
#to retrieve the best run, and thus nothing was assigned the best_run object. The question then would be: did the
#training complete successfully?

# print('fiel names {}'.format(best_run.get_file_names()))

model = best_run.register_model(model_name = 'best_model', model_path = 'outputs/model.joblib')
#, tags={"Method":"HyperDrive"}, properties={"Accuracy":best_run.get_metrics()["Accuracy"]})
#https://github.com/Azure/MachineLearningNotebooks/issues/269
#!!!!

AttributeError: 'NoneType' object has no attribute 'register_model'

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
url="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds=TabularDatasetFactory.from_delimited_files(path=url)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig
from sklearn.model_selection import train_test_split

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.


x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)


automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="Accuracy", #AUC_weighted
    training_data=ds,

    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run
run=exp.submit(config=automl_config,show_output=True)
### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
run.get_metrics()

import onnxmltools

best_run, fitted_model = run.get_output()
#https://docs.microsoft.com/en-us/azure/machine-learning/tutorial-auto-train-models

onnxmltools.utils.save_model(onnx_model, './outputs/best_automl.onnx')

In [ ]:
cpu_cluster.delete()
#https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py#delete--